In [1]:
import requests
import zipfile
import os
from bs4 import BeautifulSoup
import pandas as pd
import geopandas as gpd

- country (ctry), 
- lower layer super output area (lsoa), 
- local authority (ltla), 
- middle layer super output area (msoa), 
- output area (oa), 
- region (rgn), 
- upper tier local authority (utla)

In [7]:
def find_download_links(url):
    """
    Function to scrape the web page to find direct download links for zip files.
    """
    response = requests.get(url)
    response.raise_for_status()
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Base URL for constructing the full URL
    base_url = 'https://www.nomisweb.co.uk'

    # Find all <a> tags with href attribute containing the zip file path
    download_links = soup.find_all('a', href=lambda href: href and '.zip' in href)

    # Construct full URLs
    full_urls = [base_url + link['href'] for link in download_links]

    # Filter and return URLs that match the expected format
    return [url for url in full_urls if url.startswith('https://www.nomisweb.co.uk/output/census/2021/')]


def download_and_extract_zip(url, extract_to_folder):
    """
    function to download, extract only CSV files containing 'oa' or 'lsoa' in their names from a given zip file URL, 
    and delete the zip file, whether it's empty or not, after attempting extraction.
    """
    response = requests.get(url)
    zip_filename = os.path.join(extract_to_folder, url.split('/')[-1])
    
    with open(zip_filename, 'wb') as file:
        file.write(response.content)

    try:
        with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
            # Check if the zip file is empty
            if len(zip_ref.namelist()) == 0:
                raise ValueError(f"Zip file {zip_filename} is empty and can't be extracted.")

            # List all file names in the zip file
            all_files = zip_ref.namelist()

            # Filter out files that are not CSV or don't contain 'oa' or 'lsoa' in their names
            filtered_files = [f for f in all_files if f.endswith('.csv') and ('-oa' in f.lower() or '-lsoa' in f.lower())]

            # Extract only the filtered files
            for file in filtered_files:
                zip_ref.extract(file, extract_to_folder)
    finally:
        # Delete the zip file after extraction or if it's empty
        os.remove(zip_filename)

def download_and_extract_shapefiles(url, extract_to_folder):
    """
    Download a zip file from the URL and extract its contents to the specified folder.
    """
    try:
        # Get the file name from the URL
        zip_filename = os.path.join(extract_to_folder, url.split('/')[-1])

        # Download the zip file
        response = requests.get(url)
        response.raise_for_status()  # will raise an exception for HTTP error codes

        # Write the downloaded content to a file
        with open(zip_filename, 'wb') as file:
            file.write(response.content)

        # Extract the zip file
        with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
            zip_ref.extractall(extract_to_folder)

        print(f"Extracted {zip_filename} to {extract_to_folder}")

    except Exception as e:
        print(f"An error occurred: {e}")

    finally:
        # Delete the zip file after extraction
        if os.path.exists(zip_filename):
            os.remove(zip_filename)
            print(f"Deleted {zip_filename}")

In [8]:
# URL of the page to scrape
scrape_url = 'https://www.nomisweb.co.uk/sources/census_2021_bulk'

# Directory for downloads and extractions
extract_to_folder = os.path.join(os.getcwd(), "data")
os.makedirs(extract_to_folder, exist_ok=True)

# Get all download links
try:
    zip_file_urls = find_download_links(scrape_url)

    # Download and extract each zip file
    for url in zip_file_urls:
        try:
            download_and_extract_zip(url, extract_to_folder)
        except Exception as e:
            print(f"Error downloading or extracting {url}: {e}")
except Exception as e:
    print(f"Error scraping {scrape_url}: {e}")

print("Download and extraction completed.")

Error downloading or extracting https://www.nomisweb.co.uk/output/census/2021/census2021-ts079.zip: File is not a zip file
Error downloading or extracting https://www.nomisweb.co.uk/output/census/2021/census2021-ts079-extra.zip: File is not a zip file
Error downloading or extracting https://www.nomisweb.co.uk/output/census/2021/census2021-ts070-extra.zip: File is not a zip file
Error downloading or extracting https://www.nomisweb.co.uk/output/census/2021/census2021-ts077-extra.zip: File is not a zip file
Error downloading or extracting https://www.nomisweb.co.uk/output/census/2021/census2021-ts078-extra.zip: File is not a zip file
Error downloading or extracting https://www.nomisweb.co.uk/output/census/2021/census2021-ts037asp-extra.zip: File is not a zip file
Error downloading or extracting https://www.nomisweb.co.uk/output/census/2021/census2021-ts038asp-extra.zip: File is not a zip file
Error downloading or extracting https://www.nomisweb.co.uk/output/census/2021/census2021-ts039asp

In [19]:
# URL of shape file
shapes_url = 'https://data.london.gov.uk/download/statistical-gis-boundary-files-london/9ba8c833-6370-4b11-abdc-314aa020d5e0/statistical-gis-boundaries-london.zip'
try:
    download_and_extract_shapefiles(shapes_url, extract_to_folder)
except Exception as e:
    print(f"Error downloading or extracting {shapes_url}: {e}")

In [13]:
def merge_csv_files(directory, file_suffix):
    """
    Merge all CSV files in the directory that end with a specific suffix.
    Use 'date' and 'geography code' for merging, and include all columns from all files.
    Skip over any files that are empty.
    """
    merged_df = None
    for filename in os.listdir(directory):
        if filename.endswith(file_suffix + '.csv'):
            filepath = os.path.join(directory, filename)
            try:
                df = pd.read_csv(filepath)
                df.drop(columns=['geography'], inplace=True, errors='ignore')
            except Exception as e:
                print(f"Warning: The file {filename} is empty and will be skipped.")
                continue

            if merged_df is None:
                merged_df = df
            else:
                merged_df = pd.merge(merged_df, df, on=['date', 'geography code'], how='outer')

    # Check for duplicates and remove them
    if merged_df is not None and merged_df.duplicated().any():
        num_duplicates = merged_df.duplicated().sum()
        print(f"Number of duplicates in {file_suffix} files: {num_duplicates}")
        merged_df.drop_duplicates(inplace=True)
        
    return merged_df


# Merge and save 'oa' files
merged_oa_df = merge_csv_files(extract_to_folder, '-oa')
merged_oa_df = merged_oa_df.rename(columns={'geography code':'OA11CD'})
merged_oa_df.to_csv(os.path.join(extract_to_folder, 'combined_oa.csv'), index=False)

# Merge and save 'lsoa' files
merged_lsoa_df = merge_csv_files(extract_to_folder, 'lsoa')
merged_lsoa_df = merged_lsoa_df.rename(columns={'geography code':'LSOA11CD'})
merged_lsoa_df.to_csv(os.path.join(extract_to_folder, 'combined_lsoa.csv'), index=False)

In [12]:
# Find columns with nulls and count them
null_counts = merged_oa_df.isnull().sum()
columns_with_nulls = null_counts[null_counts > 0]

# Print columns with nulls and their counts
for column, count in columns_with_nulls.items():
    print(f"Column '{column}' has {count} nulls")

Column 'Welsh language skills: Total: All usual residents aged 3 years and over' has 178605 nulls
Column 'Welsh language skills: Can understand spoken Welsh only' has 178605 nulls
Column 'Welsh language skills: Can speak, read and write Welsh' has 178605 nulls
Column 'Welsh language skills: Can speak but cannot read or write Welsh' has 178605 nulls
Column 'Welsh language skills: Can speak and read but cannot write Welsh' has 178605 nulls
Column 'Welsh language skills: Can read but cannot speak or write Welsh' has 178605 nulls
Column 'Welsh language skills: Can write but cannot speak or read Welsh' has 178605 nulls
Column 'Welsh language skills: Can read and write but cannot speak Welsh' has 178605 nulls
Column 'Welsh language skills: Can speak and other combinations of skills in Welsh' has 178605 nulls
Column 'Welsh language skills: No skills in Welsh' has 178605 nulls
Column 'Welsh speaking ability: Total: All usual residents aged 3 years and over' has 178605 nulls
Column 'Welsh speak

In [17]:
oa_shape = gpd.read_file(f"{os.getcwd()}\\data\\statistical-gis-boundaries-london\\ESRI\\OA_2011_London_gen_MHW.shp")
oa_shape = oa_shape[['OA11CD','USUALRES','HHOLDRES', 'COMESTRES', 'POPDEN', 'HHOLDS', 'AVHHOLDSZ', 'geometry']]

lsoa_shape = gpd.read_file(f"{os.getcwd()}\\data\\statistical-gis-boundaries-london\\ESRI\\LSOA_2011_London_gen_MHW.shp")
lsoa_shape = lsoa_shape[['LSOA11CD','USUALRES', 'HHOLDRES', 'COMESTRES', 'POPDEN', 'HHOLDS', 'AVHHOLDSZ', 'geometry']]

oa_shape.shape, len(oa_shape.OA11CD.unique()), lsoa_shape.shape, len(lsoa_shape.LSOA11CD.unique())

((25053, 8), 25053, (4835, 8), 4835)

In [18]:
def merge_and_save_data(df1, df2, join_column, output_folder, merged_filename, excluded_filename):
    """
    Merge two DataFrames on a specified column, save the merged result, 
    and also save the data from each DataFrame not included in the merge.

    :param df1: First DataFrame.
    :param df2: Second DataFrame.
    :param join_column: Column name on which to join.
    :param output_folder: Folder to save the output files.
    :param merged_filename: Filename for the merged data.
    :param excluded_filename: Filename for the data excluded from the merge.
    """
    # inner join
    merged_df = pd.merge(df1, df2, on=join_column, how='inner')
    merged_df.to_csv(os.path.join(output_folder, merged_filename), index=False)

    # save excluded data
    excluded_df1 = df1[~df1[join_column].isin(merged_df[join_column])]
    excluded_df2 = df2[~df2[join_column].isin(merged_df[join_column])]
    excluded_df = pd.concat([excluded_df1, excluded_df2], ignore_index=True)
    excluded_df.to_csv(os.path.join(output_folder, excluded_filename), index=False)

    print(f"Merge complete. Files saved. Merged data shape is: {merged_df.shape}, excluded data shape is: {excluded_df.shape}")
    print(f"\nCencus data's geocode is {len(set(df1[join_column].unique())-set(df2[join_column].unique()))} more than 2011 shape file's geocode.")

oa_df = merged_oa_df.drop('date', axis=1)    
lsoa_df = merged_lsoa_df.drop('date', axis=1)

# for OA data
merge_and_save_data(
    oa_df, 
    oa_shape, 
    'OA11CD', 
    extract_to_folder, 
    'merged_inner_oa.csv', 
    'excluded_oa_data.csv'
)

# for LSOA data
merge_and_save_data(
    lsoa_df, 
    lsoa_shape, 
    'LSOA11CD', 
    extract_to_folder, 
    'merged_inner_lsoa.csv', 
    'excluded_lsoa_data.csv'
)

Merge complete. Files saved. Merged data shape is: (23913, 497), excluded data shape is: (166107, 497)

Cencus data's geocode is 164967 more than 2011 shape file's geocode.
Merge complete. Files saved. Merged data shape is: (4659, 950), excluded data shape is: (31189, 950)

Cencus data's geocode is 31013 more than 2011 shape file's geocode.


In [2]:
def zip_files(folder, filenames, zip_name):
    """
    Zip multiple files into a single zip archive.

    :param folder: The folder where files are located.
    :param filenames: A list of filenames to be zipped.
    :param zip_name: The name of the output zip file.
    """
    with zipfile.ZipFile(os.path.join(folder, zip_name), 'w') as zipf:
        for file in filenames:
            file_path = os.path.join(folder, file)
            if os.path.exists(file_path):
                zipf.write(file_path, arcname=file)
        print(f"Files zipped into {zip_name}")


In [3]:
extract_to_folder = os.path.join(os.getcwd(), "data")

# Zipping OA files
oa_files = ['merged_inner_oa.csv', 'excluded_oa_data.csv']
zip_files(extract_to_folder, oa_files, 'oa.zip')

# Zipping LSOA files
lsoa_files = ['merged_inner_lsoa.csv', 'excluded_lsoa_data.csv']
zip_files(extract_to_folder, lsoa_files, 'lsoa.zip')


Files zipped into oa.zip
Files zipped into lsoa.zip


In [14]:
def check_consistency(folder_path, feature):
    """
    Check consistency for all CSV files in the given folder.

    :param folder_path: Path to the folder containing CSV files.
    :param feature: Name of the column to check.
    :return: None
    """
    inconsistencies = []
    rlt = {}

    # Iterate through all files in the folder
    for file in os.listdir(folder_path):
        if file.endswith('.csv'):
            file_path = os.path.join(folder_path, file)
            try:
                # Read the CSV file
                df = pd.read_csv(file_path)

                # Check if the spatial column exists
                if feature in df.columns:
                    # Get unique values in the spatial column
                    unique_values = df[feature].unique()

                    # Store the count of unique values for each file
                    rlt[file] = len(unique_values)
                else:
                    inconsistencies.append(f"{file}: Spatial column '{feature}' not found.")

            except Exception as e:
                inconsistencies.append(f"{file}: Error reading file - {e}")

    # Compare resolutions across files
    if len(rlt) > 1:
        ref_ = next(iter(rlt.values()))  # Reference resolution from the first file
        for file, res in rlt.items():
            if res != ref_:
                inconsistencies.append(f"{file}: inconsistency (Expected: {ref_}, Found: {res})")

    if inconsistencies:
        print("Inconsistencies found:")
        for issue in inconsistencies:
            print(issue)
    else:
        print("All files are consistent.")

# Example usage
check_consistency(folder_path=extract_to_folder, feature='geography code')

Inconsistencies found:
census2021-ts010-lsoa.csv: Error reading file - No columns to parse from file
census2021-ts010-oa.csv: Error reading file - No columns to parse from file
combined_lsoa.csv: Spatial column 'geography code' not found.
combined_oa.csv: Spatial column 'geography code' not found.
census2021-ts001-oa.csv: inconsistency (Expected: 35672, Found: 188880)
census2021-ts002-oa.csv: inconsistency (Expected: 35672, Found: 188880)
census2021-ts003-oa.csv: inconsistency (Expected: 35672, Found: 188880)
census2021-ts004-oa.csv: inconsistency (Expected: 35672, Found: 188880)
census2021-ts005-oa.csv: inconsistency (Expected: 35672, Found: 188880)
census2021-ts006-oa.csv: inconsistency (Expected: 35672, Found: 188880)
census2021-ts007a-oa.csv: inconsistency (Expected: 35672, Found: 188880)
census2021-ts008-oa.csv: inconsistency (Expected: 35672, Found: 188880)
census2021-ts011-oa.csv: inconsistency (Expected: 35672, Found: 188880)
census2021-ts015-oa.csv: inconsistency (Expected: 35